# Model for Role Requirements

## Import Libraries

In [87]:
import pandas
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
import numpy as np



## Get Profile from an Employee

In [88]:
# Get employee's profile

# List skills that the person has and also the list of interests
person_skills = ['Python','MySQL','SQL','Java','Microsoft Azure','Kubernetes','Docker']
person_interests = ['SQL','Python']
level_requirement = 'Architect' # not used in this match

# Get list of all skills and interests
person_all_skills = person_skills + person_interests


## Transform Role Requirements Dataset

In [89]:
# Read source dataset that contains role requirements
df = pandas.read_csv(filepath_or_buffer="data/skill2vec_50k.csv", sep=",", encoding="latin1", header = None)
df = df.fillna('')
df

/Users/vanessamueller/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,

,0,1,2,3,4,5,6,7,8,9,...,951,952,953,954,955,956,957,958,959,960
0,125720,HR Executive,screening,selection,Interview,HR,Recruiter,IT Recruiter,Sourcing,recruitment executive,...,,,,,,,,,,
1,112708,Special Teacher,Teaching,Education,,,,,,,...,,,,,,,,,,
2,115226,consulting,fresher,IT helpdesk,Techincal Troubleshooting,international voice,international BPO,technical support,outsourcing,call center,...,,,,,,,,,,
3,19805,diploma,machining,cnc m,mould,conventional machines,die making,knowledge,tool,cipet,...,,,,,,,,,,
4,80208,Compensation,Benefits,HR Functions,Alm,Payroll,ESS,Core HR,QC,QA,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,27374,Chief Engineer,,,,,,,,,...,,,,,,,,,,
49996,88457,Receptionist Activities,Front Desk,front office,front desk executive,front office executive,receptionist,reception,,,...,,,,,,,,,,
49997,34590,SQL Queries,Log Analysis,Hardware Networking,People Leadership,Technical Skills,Unix,Oracle,Service Delivery Management,Database Administration,...,,,,,,,,,,
49998,86171,Quality Analyst,,,,,,,,,...,,,,,,,,,,


In [90]:
#df = df.sample(1000)

# Join all skills in a same skill column
df["Skills"] = df[1]
for x in range(2,961):
    try:
        df["Skills"] = df["Skills"] + ";" + df[x]
    except:
        errors = df[x]
print(errors)

df["id"] = df[0]
df = df[["id", "Skills"]]
df


0         
1         
2         
3         
4         
        ..
49995     
49996     
49997     
49998     
49999     
Name: 660, Length: 50000, dtype: object


,id,Skills
0,125720,HR Executive;screening;selection;Interview;HR;...
1,112708,Special Teacher;Teaching;Education;;;;;;;;;;;;...
2,115226,consulting;fresher;IT helpdesk;Techincal Troub...
3,19805,diploma;machining;cnc m;mould;conventional mac...
4,80208,Compensation;Benefits;HR Functions;Alm;Payroll...
...,...,...
49995,27374,Chief Engineer;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...
49996,88457,Receptionist Activities;Front Desk;front offic...
49997,34590,SQL Queries;Log Analysis;Hardware Networking;P...
49998,86171,Quality Analyst;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;...


In [91]:
# Filter for only roles that mention any of the relevant skills
#synonym_skill = skills_df['synonym']
#def find_index(arr, val):
#    for i in range(len(arr)):
#        if arr[i] == val:
#            return i
#    return -1

#total_skills = []

#count = 0
#for skill in person_all_skills:
#    total_skills.append(df["Skills"].apply(lambda x: 1 if skill in df['Skills'][count] or synonym_skill[find_index(person_all_skills, skill)] in df['Skills'][count] else 0))

    
#total_skills
##df = df.loc[df['CountSkills'] > 0]


##is_present = any(df['Skills'].isin(person_all_skills))
##is_present


In [92]:
# Get list of all skills available to be selected - Source: Stack Overflow Survey questions
skills_df = pandas.read_csv(filepath_or_buffer="data/skill-list.csv", sep=",", encoding="latin1")
skills_df

df = df.sample(100)

# Get only positions with skills that are mapped
skills = skills_df['skill']

# Get only skills that are relevant for the person
#skills = person_all_skills

# Account for synonyms
synonym_skill = skills_df['synonym']
def find_index(arr, val):
    for i in range(len(arr)):
        if arr[i] == val:
            return i
    return -1

for skill in skills:
    df[skill] = df["Skills"].apply(lambda x: 1 if skill.lower() in [item.lower() for item in x.split(';')] or synonym_skill[find_index(skills, skill)] in [item.lower() for item in x.split(';')] else 0)
df


,id,Skills,APL,Assembly,Bash/Shell,C,C#,C++,Clojure,COBOL,...,Flow,Homebrew,Kubernetes,npm,Pulumi,Puppet,Terraform,Unity 3D,Unreal Engine,Yarn
37366,12659,acconts incharge;acconts head;accounts supervi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33855,72033,Medical coding jobs;Medical coder;Medical codi...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30517,76224,vashi;chembur;sion;inbound;BAF;Voice;customer ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29584,89576,Accounts Finance tally invoicing...;Account Pa...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27951,72284,healthcare;life sciences;mr;pharma;pharmaceuti...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26630,111652,PL SQL Developer;Oracle;PL SQL;System Analyst;...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18620,99630,Rest;restful;integration framework;camel;Core ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29952,49447,Finance Head;Financial Analysis;Finance;Accoun...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22103,39737,labview;labview RT;CLAD;CLD;Real time;automati...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [93]:
# Remove rows that don't have any of the person's skills or interests
df = df.loc[(df[person_all_skills] != 0).any(axis=1)]

# Remove columns where all values are 0
df = df.loc[:, (df != 0).any(axis=0)]
df

,id,Skills,HTML/CSS,Java,JavaScript,Python,Ruby,SQL,MySQL,Oracle,ASP.NET,Django,.NET,Spring
42166,105696,Java Programming;J2Ee Spring;Sql Coding;Electr...,1,1,0,0,0,1,0,1,1,0,1,0
15745,15155,scripts;SQL;Oracle PL;Unix;Websphere;Productio...,0,0,0,0,0,1,0,1,0,0,0,0
1958,14577,Enterprise Architect;Tool Design;Java;Android ...,0,1,0,0,0,0,0,0,0,0,0,0
43949,68679,Technical Leadership;Java;Programming;Coding;S...,0,1,0,0,0,0,0,0,0,0,0,0
20513,63209,Front End;HTML;CSS;Javascript;MySQL;Ruby;Web T...,1,0,1,0,1,0,1,0,0,0,0,0
20293,21397,Front End;Python;Django;Automation Testing;sof...,0,0,0,1,0,0,0,0,0,1,0,0
18620,99630,Rest;restful;integration framework;camel;Core ...,0,1,0,0,0,0,0,0,0,0,0,1


In [94]:
# Get list of skills in this dataset
skills = columns = list(df.drop(columns=['id','Skills']).columns)

# Replicate skills columns to interests
for skill in skills:
    df[skill+'-Want'] = df[skill]
df

# Remove unused Skills column
df = df.drop(columns=["Skills"])
df

,id,HTML/CSS,Java,JavaScript,Python,Ruby,SQL,MySQL,Oracle,ASP.NET,...,JavaScript-Want,Python-Want,Ruby-Want,SQL-Want,MySQL-Want,Oracle-Want,ASP.NET-Want,Django-Want,.NET-Want,Spring-Want
42166,105696,1,1,0,0,0,1,0,1,1,...,0,0,0,1,0,1,1,0,1,0
15745,15155,0,0,0,0,0,1,0,1,0,...,0,0,0,1,0,1,0,0,0,0
1958,14577,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43949,68679,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20513,63209,1,0,1,0,1,0,1,0,0,...,1,0,1,0,1,0,0,0,0,0
20293,21397,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
18620,99630,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [95]:
# Create an array for the person
size = len(df.drop(columns='id').columns)
person = np.zeros(shape=(1, size))

# Set value to skills and interests
count = 0
for skill in list(df.drop(columns='id').columns):
    if ("Want" in skill):
        skill = skill.replace("-Want", "")
        person[0][count] = 10 if skill in person_interests else 0
        print(skill, " - ", person[0][count])
    else:
        if skill in person_skills and skill in person_interests:
            person[0][count] = 10
        elif skill in person_skills and skill not in person_interests:
            person[0][count] = 5
        elif skill not in person_skills and skill in person_interests:
            person[0][count] = 2
        else:
            person[0][count] = 0
        #print(skill, " - ", person[0][count])
    count = count +1

person



HTML/CSS  -  0.0
Java  -  0.0
JavaScript  -  0.0
Python  -  10.0
Ruby  -  0.0
SQL  -  10.0
MySQL  -  0.0
Oracle  -  0.0
ASP.NET  -  0.0
Django  -  0.0
.NET  -  0.0
Spring  -  0.0


array([[ 0.,  5.,  0., 10.,  0., 10.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., 10.,  0., 10.,  0.,  0.,  0.,  0.,  0.,  0.]])

## KMeans to clusterize Roles

In [96]:
# Set number of clusters
number_clusters = df.drop(columns=["id"]).drop_duplicates().count()[0]

# Group roles into clusters
model = KMeans(n_clusters=number_clusters, random_state=42).fit(df.drop(columns=["id"]))
clusters = model.labels_
#center = model.cluster_centers_

# Assign clusters to roles
df_clusters = df
df_clusters["cluster"] = clusters
df_clusters

,id,HTML/CSS,Java,JavaScript,Python,Ruby,SQL,MySQL,Oracle,ASP.NET,...,Python-Want,Ruby-Want,SQL-Want,MySQL-Want,Oracle-Want,ASP.NET-Want,Django-Want,.NET-Want,Spring-Want,cluster
42166,105696,1,1,0,0,0,1,0,1,1,...,0,0,1,0,1,1,0,1,0,3
15745,15155,0,0,0,0,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,2
1958,14577,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
43949,68679,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
20513,63209,1,0,1,0,1,0,1,0,0,...,0,1,0,1,0,0,0,0,0,1
20293,21397,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,4
18620,99630,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [97]:
# Check the roles dataset for the skills and interests of the person
roles_df = df_clusters.sort_values("cluster")
list_of_skills = [x for x in person_all_skills if x in list(roles_df.columns.values)]
roles_df[list_of_skills]


,Python,MySQL,SQL,Java,SQL,Python
18620,0,0,0,1,0,0
20513,0,1,0,0,0,0
15745,0,0,1,0,1,0
42166,0,0,1,1,1,0
20293,1,0,0,0,0,1
1958,0,0,0,1,0,0
43949,0,0,0,1,0,0


## KNN to find the Best Matches

In [98]:
# Find the one role cluster closer to the person
knn = KNeighborsClassifier(n_neighbors=1)

knn.fit(roles_df.drop(columns=["id", "cluster"]), roles_df['cluster'])

prediction = knn.predict(person)

prediction

array([3], dtype=int32)

## Get list of the matched Roles

In [99]:
# Role that was found in the match
roles_df.loc[roles_df['cluster'] == prediction[0]]
#roles_df.loc[roles_df['id'] == 9271]



,id,HTML/CSS,Java,JavaScript,Python,Ruby,SQL,MySQL,Oracle,ASP.NET,...,Python-Want,Ruby-Want,SQL-Want,MySQL-Want,Oracle-Want,ASP.NET-Want,Django-Want,.NET-Want,Spring-Want,cluster
42166,105696,1,1,0,0,0,1,0,1,1,...,0,0,1,0,1,1,0,1,0,3
